In [1]:
options(warn = -1)

In [2]:
library(stringr)
library(rgdal)
library(lubridate)

Loading required package: sp

rgdal: version: 1.5-23, (SVN revision 1121)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.2.1, released 2020/12/29
Path to GDAL shared files: C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/gdal
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 7.2.1, January 1st, 2021, [PJ_VERSION: 721]
Path to PROJ shared files: C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/proj
PROJ CDN enabled: TRUE
Linking to sp version:1.4-5
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rgdal_show_exportToProj4_warnings"="none") before loading rgdal.
Overwritten PROJ_LIB was C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/proj


Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union




In [3]:
path <- "T:/MPO/Bike&Ped/BikeCounting/StoryMap/BikeShare/Output"

In [4]:
data <- read.csv(paste0(path, "/Bike_Share_Trips.csv"))

In [5]:
data$Date <- as.Date(data$Date, "%Y-%m-%d")

In [6]:
unique(year(data$Date))

[1] 2021 2022

In [7]:
head(data)

,RouteID,BikeID,UserID,Location,Latitude,Longitude,Date,Time,PathID,Distance,Minutes,OrgDst,Weekday,Month,Season,SeasonOrder,WeekdayOrder
,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<date>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
1,13713102,19521,783398,Monroe St & Blair Blvd,44.05141,-123.1049,2021-01-01,01:11,Monroe St & Blair Blvd - 15th & Ferry,1.55,11.000000,Origin,Friday,January,Winter,4,5
2,13713106,19584,1510848,PeaceHealth University District,44.04736,-123.0822,2021-01-01,01:52,PeaceHealth University District - PeaceHealth University District,1.17,20.550000,Origin,Friday,January,Winter,4,5
3,13713112,19737,911293,12th & Willamette,44.04608,-123.0929,2021-01-01,03:16,"12th & Willamette - 17th & Pearl, East side",0.49,4.916667,Origin,Friday,January,Winter,4,5
4,13713256,19737,766725,"17th & Pearl, East side",44.04109,-123.0900,2021-01-01,09:36,"17th & Pearl, East side - 17th & Pearl, East side",2.29,23.100000,Origin,Friday,January,Winter,4,5
5,13713355,19761,1726219,15th & Ferry,44.04355,-123.0852,2021-01-01,11:09,15th & Ferry - 15th & Ferry,0.01,1.100000,Origin,Friday,January,Winter,4,5
6,13713360,19711,1174607,Valley River,44.06523,-123.1037,2021-01-01,11:13,Valley River - Valley River,7.60,36.500000,Origin,Friday,January,Winter,4,5


In [8]:
dim(data)

[1] 173120     17

In [9]:
x <- unique(data$Time)[1]

In [10]:
x

[1] "01:11"

In [11]:
convert_time_to_hour <- function(x){
    s <- unlist(str_split(x, ":"))
    return(as.numeric(s[1])+as.numeric(s[2])/60)
}

In [12]:
data$Hour <- sapply(data$Time, function(x) convert_time_to_hour(x))

In [13]:
get_parts_of_day <- function(x){
    if(x >= 21 | x <= 4){
        d <- "Night"
    }else if(x > 17 & x < 21){
        d <- "Evening"
    }else if(x > 12 & x <= 17){
#        if(x > 12 & x < 14){
#            d <- "Early Afternoon"
#        }else if(x > 16 & x <= 17){
#            d <- "Late Afternoon"
#        }else{
#            d <- "Afternoon"
#        }
        d <- "Afternoon"
    }else{
#        if(x > 4 & x < 8){
#            d <- "Early Morning"
#        }else if(x >= 11 & x <= 12){
#            d <- "Late Morning"
#        }else{
#            d <- "Morning"
#        }
        d <- "Morning"
    }
    return(d)
}

In [14]:
get_parts_of_day(convert_time_to_hour(x))

[1] "Night"

In [15]:
data$DayPart <- sapply(data$Hour, function(x) get_parts_of_day(x))

In [16]:
names(data)

[1] "RouteID"      "BikeID"       "UserID"       "Location"     "Latitude"    
 [6] "Longitude"    "Date"         "Time"         "PathID"       "Distance"    
[11] "Minutes"      "OrgDst"       "Weekday"      "Month"        "Season"      
[16] "SeasonOrder"  "WeekdayOrder" "Hour"         "DayPart"

In [17]:
agg_data <- function(df=data, by="Origin"){
    dt <- df[df$OrgDst == by,]
    aggdt <- aggregate(x=list(NoTrips=dt$RouteID, NoUsers = dt$UserID),
              by=list(Location = dt$Location, Longitude = dt$Longitude, Latitude=dt$Latitude, DayPart=dt$DayPart),
              FUN=function(x) length(unique(x)))
    return(aggdt)
}

In [18]:
Origin_dt <- agg_data()

In [19]:
Destination_dt <- agg_data(df=data, by="Destination")

In [20]:
df2spdf <- function(df, lon_col_name, lat_col_name, trans = TRUE){
  lonlat <- sp::CRS("+proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0")
  lon_col_no <- which(names(df)==lon_col_name)
  lat_col_no <- which(names(df)==lat_col_name)
  xy <- data.frame(df[,c(lon_col_no,lat_col_no)])
  coordinates(xy) <- c(lon_col_name, lat_col_name)
  proj4string(xy) <- lonlat
  spdf <- sp::SpatialPointsDataFrame(coords = xy, data = df)
  if(trans){
    spdf <- spTransform(spdf,  CRS("+proj=merc +a=6378137 +b=6378137 +lat_ts=0 +lon_0=0 +x_0=0 +y_0=0 +k=1 +units=m +nadgrids=@null +wktext +no_defs +type=crs"))
  }
  return(spdf)
}

In [21]:
Origin_spdf <- df2spdf(Origin_dt, 'Longitude', 'Latitude')

In [22]:
Destination_spdf <- df2spdf(Destination_dt, 'Longitude', 'Latitude')

In [23]:
writeOGR(Origin_spdf, dsn=path, layer="Origin_Bike_Share_DayPart", driver="ESRI Shapefile", overwrite_layer=TRUE)

In [24]:
writeOGR(Destination_spdf, dsn=path, layer="Destination_Bike_Share_DayPart", driver="ESRI Shapefile", overwrite_layer=TRUE)